-----

### Onto the Scraping!
#### gameplan -

While Spotify's documentation doesn't directly state what their call rate limit is, a little digging seems to indicate a few hundred calls per minute should go through without issue.  If needed, we'll play around with our call delays and see if/when we get bumped.

Rather than query each song individually, we can query the artist and receive batches of 50 songs and their spotify uids.  (The api call limits us to 50 results per call)  The call allows for an offset to where the return data picks up, so if an artist has 300 tracks, we can call the same artist and offset by 50 each consecutive time until we get the track ids needed.

We'll query an artist, log their artist uid and the 50 tracks returned from the first call.  If those results include all the tracks we need, then we progress to the next artist, otherwise we call again with the offset increased by 50.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlalchemy as sa
import psycopg2
import json
import time
import datetime
import os
import sys
import csv
import uuid
import sqlite3

from tqdm import tqdm

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from difflib import get_close_matches, SequenceMatcher
from dotenv import dotenv_values

In [2]:
cfg = dotenv_values("../.env")

PG_user = cfg['PGSQL_USER']
PG_pw = cfg['PGSQL_PW']
PG_host = cfg['PGSQL_HOST']
PG_db = cfg['PGSQL_DB']

SPOT_id = cfg['SPOTIPY_CLIENT_ID']
SPOT_secret = cfg['SPOTIPY_CLIENT_SECRET']

In [3]:
processed_data_path = '../data/processed/'

# df_songs = pd.read_csv(processed_data_path + 'df_songs_norm.csv')
df_playlists = pd.read_csv(processed_data_path + 'df_playlists.csv', index_col=0)

with open(processed_data_path + 'artist-songs-trimmed.json', 'r') as f:
    artist_songs = json.load(f)
list_artists = list(artist_songs.keys())
list_artists.sort()

In [4]:
df_playlists.head()

,artist_name,track_name,playlist_uuid,user_id
0,Elvis Costello,(the Angels Wanna Wear My) Red Shoes,58048995723,9cc0cfd4d7d7885102480dd99e7a90d6
1,Elvis Costello & The Attractions,"(What's So Funny 'Bout) Peace, Love And Unders...",58048995723,9cc0cfd4d7d7885102480dd99e7a90d6
2,Elvis Costello & The Attractions,Accidents Will Happen,58048995723,9cc0cfd4d7d7885102480dd99e7a90d6
3,Elvis Costello,Alison,58048995723,9cc0cfd4d7d7885102480dd99e7a90d6
4,Lissie,All Be Okay,58048995723,9cc0cfd4d7d7885102480dd99e7a90d6


In [5]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
                                        client_id=SPOT_id,
                                        client_secret=SPOT_secret)
)

starting with a single pull to review the return -

In [7]:
# starting with one
q_artist = 'Amon Amarth'
# print(len(artist_songs[q_artist]))
results = sp.search(q=f'artist:{q_artist}', offset=0, limit=50)
results = results['tracks']

with open('result_view.json', 'w') as f:
    json.dump(results, f, indent=4)

After a bit of poking around this data and how the api response is structured, the following function will create a dictionary containing artists, their ids, and a list of songs with the song ids.

In [6]:
def scrape_artist(query_name, call_count=0, offset=0):
    '''
    Description:
        query the spotify API for up to 1k song returns 
    Arguments:
        query_name: str, artist's name being queried
        call_count: int, count of how many times api has been called this session
        offset: int, used to declare starting point for subsequent calls
    Returns:
        compiled_return, call_count: dict, int
    '''

    compiled_return = {}

    while True:
        call_count += 1
        results = sp.search(q=f'artist:{query_name}', offset=offset, limit=50)
        results = results['tracks']

        for track in results['items']:
            artist_id = track['artists'][0]['id']
            artist_name = track['artists'][0]['name']
            track_id = track['id']
            track_name = track['name']

            if artist_id in compiled_return:
                compiled_return[artist_id]['tracks'][track_id] = track_name
            else:
                compiled_return[artist_id] = {'artist_name': artist_name,
                                              'tracks': {track_id:track_name}}

        if not results['next'] or offset == 950:
            break

        else:
            offset += 50
            time.sleep(.1)
    
    return compiled_return, call_count

Test a scrape with artists starting with the letter 'B'

In [9]:
# test_artists = [i for i in list_artists if i[0].lower() == 'b']
# print(len(test_artists))
# print(test_artists[:10])
# test_scrape = {}
# missed_artists = []
# for artist in tqdm(test_artists):
#     try:
#         test_scrape[artist] = scrape_artist(artist)
#     except:
#         missed_artists.append(artist)
# with open(processed_data_path + 'scrapped-b.json', 'w') as f:
#     json.dump(test_scrape, f)

In [7]:
temp_artists_list = list(list_artists)
scrape_keys = [chr(i) for i in range(ord('a'), ord('z') + 1)]
scrape_dict = {}
for k in tqdm(scrape_keys):
    curr_list = [i for i in list_artists if i[0].lower() == k]
    scrape_dict[k] = curr_list
    temp_artists_list = [i for i in temp_artists_list if i not in curr_list]
scrape_dict['_'] = temp_artists_list

100%|██████████| 26/26 [01:03<00:00,  2.44s/it]


In [8]:
completed_keys = [chr(i) for i in range(ord('a'), ord('k') + 1)]
for i in completed_keys:
    try:
        del scrape_dict[i]
    except:
        pass
check_count = 0
for k, v in scrape_dict.items():
    check_count += len(v)
print(f'\ntotal number of artists: {len(list_artists)}')
print(f'\nstill to capture: {check_count}')
print(f'\npercent complete: {1 -(check_count / len(list_artists)):.2%}')


total number of artists: 61078

still to capture: 31818

percent complete: 47.91%


In [160]:
# del scrape_dict['j']
# last_idx = list(scrape_dict['k']).index('Klaus Schulze')
# drop_list = list(scrape_dict['k'])[:last_idx + 1]
# scrape_dict['k'] = [i for i in scrape_dict['k'] if i not in drop_list]

In [9]:
missed_artists = []
call_count = 0

for curr_key in scrape_dict.keys():

    print(f'\nstarting - {curr_key}')

    curr_list = scrape_dict[curr_key]
    curr_scrape = {}

    for artist in tqdm(curr_list):
        try:
            scrape_return, call_count = scrape_artist(artist, call_count)
            curr_scrape[artist] = scrape_return
        except:
            missed_artists.append(artist)
        if call_count >= 21950:
            print(f'ending early with call_count: {call_count}')
            print(f'last artist captured: {artist}')
            break
    
    # check to see if we've captured any of this letter before, if so merge the data
    f_path = processed_data_path + f'scraped/{curr_key}.json'
    if os.path.exists(f_path):
        with open(f_path, 'r') as f:
            temp = json.load(f)
            merge_dict = {**temp, **curr_scrape}
        with open(f_path, 'w') as f:
            json.dump(merge_dict, f)
    else:
        with open(f_path, 'w') as f:
            json.dump(curr_scrape, f)
    
    if call_count >= 21950:
        break
    print(f'\nfinished - {curr_key}')


starting - l


100%|██████████| 3241/3241 [1:45:57<00:00,  1.96s/it]  



finished - l

starting - m


  1%|          | 52/4595 [02:37<3:49:48,  3.04s/it]

ending early with call_count: 21958
last artist captured: MAM


-----
-----
-----

to query the spotify api -

`query = f'artist:{q_artist} + track:{q_track}', offset=0, limit=50'`

`results = sp.search(q=query)`

-----

#### postgres tables

__artists__
| **artist_id** | artist_name   |
| :----:        |    :----:     |
| `artist_id`   | `name_string` |
| `artist_id`   | `name_string` |
| etc...        | ...           |

__tracks__
| **track_id**          | track_name    | *`artist_id`*   | song_feature1 | song_feature2 | etc. |
| :----:       |    :----:     |    :----:   |    :----:     |    :----:     |:----:|
| `track_id`   | `name_string` | `artist_id` | `feature1val` | `feature2val` | ...  |
| `track_id`   | `name_string` | `artist_id` | `feature1val` | `feature2val` | ...  |
| etc...       | ...           | ...         |    ...        |     ...       | ...  | 

<br><br>
Our playlist's only relationship is the many to many with `track_id`, so we will
just use this as the junction table!

__playlists__
| **playlist_id**         | playlist_id   | *`track_id`*    |
| :----:                  |    :----:     |   :----:        |
| `concat playlist&track` | `playlist_id` | `track_id`      |
| `concat playlist&track` | `playlist_id` | `track_id`      |
| etc...                  | ...           |                 |

In [120]:
# # reset

# conn = psycopg2.connect(
#     host=PG_host,
#     database=PG_db,
#     user=PG_user,
#     password=PG_pw
# )

# queries = (
#     '''
#     DROP TABLE IF EXISTS playlists
#     ''',
#     '''
#     DROP TABLE IF EXISTS tracks
#     ''',
#     '''
#     DROP TABLE IF EXISTS artists
#     '''
# )

# try:
#     crsr = conn.cursor()
#     for query in queries:
#         crsr.execute(query)
#     crsr.close()
#     conn.commit()
# except(Exception, psycopg2.DatabaseError) as error:
#     print(error)


In [121]:
queries = (
    '''
    CREATE TABLE IF NOT EXISTS artists (
        artist_id VARCHAR(255) PRIMARY KEY,
        artist_name VARCHAR(255)
    )
    ''',
    '''
    CREATE TABLE IF NOT EXISTS tracks (
        track_id VARCHAR(255) PRIMARY KEY,
        track_name VARCHAR(255),
        artist_id VARCHAR(255),
        FOREIGN KEY (artist_id) REFERENCES artists (artist_id)
    )
    ''',
    '''
    CREATE TABLE IF NOT EXISTS playlists (
        uid VARCHAR(255) PRIMARY KEY,
        playlist_id VARCHAR(255),
        track_id VARCHAR(255),
        FOREIGN KEY (track_id) REFERENCES tracks (track_id)
    )
    '''
)

try:
    crsr = conn.cursor()
    for query in queries:
        crsr.execute(query)
    crsr.close()
    conn.commit()
except(Exception, psycopg2.DatabaseError) as error:
    print(error)

In [37]:
with open(processed_data_path + 'scraped/a.json', 'r') as f:
    test_letter = json.load(f)

In [77]:
conn = psycopg2.connect(
    host=PG_host,
    database=PG_db,
    user=PG_user,
    password=PG_pw
)

curr_artist_list = list(test_letter.keys())

In [129]:
for i in tqdm(curr_artist_list):
    current = test_letter[i]
    queries = []
    for artist_id, name_tracks in current.items():
        artist_string = name_tracks['artist_name']
        tracks = name_tracks['tracks']

        queries.append(crsr.mogrify('''
        INSERT INTO artists(artist_id, artist_name)
            VALUES (%s, %s)
        ''', (artist_id, artist_string)))

        for track_id, track_name in tracks.items():
            queries.append(crsr.mogrify('''
            INSERT INTO tracks(track_id, track_name, artist_id)
                VALUES (%s, %s, %s)
            ''', (track_id, track_name, artist_id)))
        
    
    for query in queries:
        try:
            crsr = conn.cursor()
            crsr.execute(query)
            crsr.close()
            conn.commit()
        except(Exception, psycopg2.DatabaseError) as error:
            # print(error)
            conn.rollback()
        # finally:
        #     if conn is not None:
        #         conn.close()


100%|██████████| 3912/3912 [12:20<00:00,  5.28it/s]
